**Google Data Analytics Professional Certificate Program : Capstone Project**

**Case Study1: Cyclistic bikes**


**Reshma TV**

**2022-08-29**



The **Cyclistic Case Study** is one of the capstone projects of **Google Data Analytics**, a professional certificate program offered on **Coursera**. I finished the certificate program recently, working on the case. It was a fun challenge, and I learned a lot from it.


**Scenario**

A junior data analyst working in the marketing analyst team at Cyclistic, a bike-share company in Chicago. They have two types of users:
* Members - who have annual memberships and
* Casuals - who rides with a single-ride or full-day passes. 
The director of marketing believes the company’s future success depends on maximizing the number of annual memberships. Therefore,analytics team want to understand how casual riders and annual members use Cyclistic bikes differently. From these insights, team will design a new marketing strategy to convert casual riders into annual members. But first, Cyclistic executives must approve your recommendations, so they must be backed up with compelling data insights and professional data visualizations.

To complete the case study, I followed the six phases involved in data analysis : **Ask, Prepare, Process, Analyze, Share and Act**

**Ask**

In this phase of data analysis, there are two key tasks : 
1. Identify the business task
2. Consider key stakeholders

Deliverables :
1. Problem : How do annual members and casual riders use Cyclistic bikes differently?
1. Stakeholders : The manager and executive team

**Prepare**

The data is on an AWS server where it is easily downloadable and named correctly. I have downloaded the previous 12 months data (from july 2021 to june 2022)and stored it locally for the next steps in the anlysis process. The data is under a license and is of first-party type. So it is reliable and original, comprehensive, current, and cited.

**Process**

Downloaded data set are in zipped form so unzip them and store in a subfolder for .CSV files.

The processing of data for the analysis starts here:

Firstly, Install and load all required packages and libraries in RStudio.

In [ ]:
install.packages("tidyverse")
install.packages("janitor")
library(tidyverse)
library(lubridate)
library(dplyr)
library(janitor)
library(ggplot2)

STEP 1 : IMPORTING DATA 


In [ ]:
jul_2021 <- read.csv("../csvs/202107-divvy-tripdata.csv")
aug_2021 <- read.csv("../csvs/202108-divvy-tripdata.csv")
sep_2021 <- read.csv("../csvs/202109-divvy-tripdata.csv")
oct_2021 <- read.csv("../csvs/202110-divvy-tripdata.csv")
nov_2021 <- read.csv("../csvs/202111-divvy-tripdata.csv")
dec_2021 <- read.csv("../csvs/202112-divvy-tripdata.csv")
jan_2022 <- read.csv("../csvs/202201-divvy-tripdata.csv")
feb_2022 <- read.csv("../csvs/202202-divvy-tripdata.csv")
mar_2022 <- read.csv("../csvs/202203-divvy-tripdata.csv")
apr_2022 <- read.csv("../csvs/202204-divvy-tripdata.csv")
may_2022 <- read.csv("../csvs/202205-divvy-tripdata.csv")
jun_2022 <- read.csv("../csvs/202206-divvy-tripdata.csv")



STEP 2 : WRANGLE DATA AND COMBINE INTO SINGLE DATAFRAME

Before merging make sure that the structure of all the files are the same. To inspect the dataset, you can use any of the following functions.

In [ ]:
head(jul_2021)
glimpse(`jul_2021`) 
str(`jul_2021`)

Compare the structure of the data set

In [ ]:
compare_df_cols(jul_2021, aug_2021, sep_2021, oct_2021, nov_2021, dec_2021, jan_2022, feb_2022, mar_2022, apr_2022, may_2022, jun_2022)


Combining all data into a single data frame

In [ ]:
tripdata <-
  rbind(jul_2021, aug_2021, sep_2021, oct_2021, nov_2021, dec_2021, jan_2022, feb_2022, mar_2022, apr_2022, may_2022, jun_2022 )


converting data type of columns

In [ ]:
tripdata$started_at = strptime(tripdata$started_at,"%Y-%m-%d %H:%M:%S")
tripdata$ended_at = strptime(tripdata$ended_at,"%Y-%m-%d %H:%M:%S")


create a new data frame with only the required fields

In [ ]:
tripdata_new <- tripdata %>% 
  select(ride_id, rideable_type, member_casual, start_station_id, start_station_name, started_at, end_station_id, end_station_name, ended_at)


In [ ]:
 STEP 3 : CLEAN UP AND ADD DATA TO PREPARE FOR ANALYSIS

Inspect the new table that has been created

In [ ]:
colnames(tripdata_new) #List of column names
nrow(tripdata_new)  #How many rows are in data frame?
dim(tripdata_new)  #Dimensions of the data frame
head(tripdata_new)  #See the first 6 rows of data frame.  
tail(tripdata_new) #See last 6 rows of data frame
str(tripdata_new)  #See list of columns and data types (numeric, character, etc)
summary(tripdata_new)  #Statistical summary of data. Mainly for numerics


check for the consistency in values


In [ ]:
unique(tripdata_new$rideable_type)
unique(tripdata_new$member_casual)


Add columns that list the date, month, day, year, start hour and end houra of each ride

In [ ]:
tripdata_new$date <- as.Date(tripdata_new$started_at) #The default format is yyyy-mm-dd
tripdata_new$month <- format(as.Date(tripdata_new$date), "%m")
tripdata_new$day <- format(as.Date(tripdata_new$date), "%d")
tripdata_new$year <- format(as.Date(tripdata_new$date), "%Y")
tripdata_new$day_of_week <- format(as.Date(tripdata_new$date), "%A")
tripdata_new$start_hour <- hour(tripdata_new$started_at)
tripdata_new$end_hour <- hour(tripdata_new$ended_at)


find the ride length and convert it into hh:mm:ss format


In [ ]:
tripdata_new$ride_length <- 
     hms::hms(seconds_to_period(tripdata_new$ended_at - tripdata_new$started_at))



search for bad data ie, data with ride length is negative


In [ ]:
filter(tripdata_new,ride_length < 0)


count the number of bad data


In [ ]:
nrow(filter(tripdata_new,ride_length < 0))  



Create a new dataframe after removing bad data


In [ ]:
tripdata_final <- tripdata_new[!(tripdata_new$ride_length<0),]



count number of final data for accuracy

In [ ]:
nrow(tripdata_final)


see how many observations fall under each usertype

In [ ]:
table(tripdata_final$member_casual)



STEP 4: CONDUCT DESCRIPTIVE ANALYSIS

some calculations for Descriptive analysis on ride_length

In [ ]:
mean(tripdata_final$ride_length) #straight average (total ride length / rides)
median(tripdata_final$ride_length) #midpoint number in the ascending array of ride lengths
max(tripdata_final$ride_length) #longest ride
min(tripdata_final$ride_length) #shortest ride



summary(tripdata_final$ride_length) # You can condense the four lines above to one line using summary() on the specific attribute

Compare ride length of members and casual users based on the calculations

In [ ]:
aggregate(tripdata_final$ride_length ~ tripdata_final$member_casual, FUN = mean)
aggregate(tripdata_final$ride_length ~ tripdata_final$member_casual, FUN = median)
aggregate(tripdata_final$ride_length ~ tripdata_final$member_casual, FUN = max)
aggregate(tripdata_final$ride_length ~ tripdata_final$member_casual, FUN = min)



Make days of the week in order

In [ ]:
tripdata_final$day_of_week <- ordered(tripdata_final$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))


Compare ride length of members and casual users by each day

In [ ]:
aggregate(tripdata_final$ride_length ~ tripdata_final$member_casual + tripdata_final$day_of_week, FUN = mean)


DATA VISUALIZATION

-------------------------user distribution-----------------------------------

In [ ]:
tripdata_final %>% 
  group_by(member_casual) %>% 
  summarise(number_of_rides = length(ride_id), ride_percentage = (length(ride_id) / nrow(tripdata_final)) * 100)

ggplot(aes(x = member_casual, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") + 
  labs(x="Casuals vs Members", y="Number Of Rides", title= "Casuals vs Members distribution")+
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))


-----------------------popular rideable type--------------------------------

In [ ]:
tripdata_final %>%
  filter(start_station_name != "")%>%
  group_by(start_station_name ) %>%
  count(start_station_name, sort = TRUE) %>% top_n(10) %>%
  ggplot() + geom_col(aes(x=start_station_name, y=n)) + coord_flip() +
  theme(axis.text.x = element_text(angle = 90))+
  labs(x="start_station_name", y="Number of rides",title = "Top 10 start stations") +
  scale_y_continuos(labels = comma)

-------------------popular start station based on number of rides----------------------------

In [ ]:
tripdata_final %>%
  filter(start_station_name != "")%>%
  group_by(start_station_name ) %>%
  count(start_station_name, sort = TRUE) %>% top_n(10) %>%
  ggplot() + geom_col(aes(x=start_station_name, y=n)) + coord_flip() +
  theme(axis.text.x = element_text(angle = 90))+
  labs(x="start_station_name", y="Number of rides",title = "Top 10 start stations") +
  scale_y_continuos(labels = comma)
  

--------------popular end station based on number of rides-----------------------------------

In [ ]:
tripdata_final %>%
  filter(end_station_name != "")%>%
  group_by(end_station_name ) %>%
  count(end_station_name, sort = TRUE) %>% top_n(10) %>%
  ggplot() + geom_col(aes(x=end_station_name, y=n)) + coord_flip() +
  theme(axis.text.x = element_text(angle = 90))+
  labs(x="end_station_name", y="Number of rides",title = "Top 10 end stations") +
  scale_y_continuos(labels = comma)

--------------number of rides by rider type in each month----------------

In [ ]:
tripdata_final %>% 
  group_by(member_casual, month) %>% 
  summarise(number_of_rides = n()) %>% 
  arrange(member_casual, month)  %>% 
  ggplot(aes(x = month, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(title ="Total rides by Members and Casual riders Vs. each Month") +
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))


-------------number of rides by rider type in each day----------------------

In [ ]:
tripdata_final %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>%     #creates weekday field using wday()
  group_by(member_casual, weekday) %>%            #groups by usertype and weekday
  summarise(number_of_rides = n())%>%        #calculates the number of rides
  arrange(member_casual, weekday)  %>%    # sorts
  ggplot(aes(x = weekday, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(title ="Total rides by Members and Casual riders Vs. Day of the week")+
  scale_y_continuous(labels = function(x) format(x, scientific = FALSE))


--------------number of rides by rider type in each hour---------------------

In [ ]:
tripdata_final %>% 
  group_by(member_casual, start_hour ) %>% 
  summarise(number_of_rides = n()) %>% 
  arrange(member_casual, start_hour )  %>% 
  ggplot(aes(x = start_hour, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(title ="Total rides by Members and Casual riders Vs. Hours of a day")


--------------average ride length by rider type in each month----------------

In [ ]:
tripdata_final %>% 
  group_by(member_casual, month) %>% 
  summarise(average_ride_length = mean(ride_length)) %>% 
  arrange(member_casual, month)  %>% 
  ggplot(aes(x = month, y = average_ride_length, fill = member_casual)) +
  geom_col(width=0.5, position = "dodge") + 
  labs(title ="Average ride length by Members and Casual riders Vs. Month over the year")


--------------average ride length by rider type in each day of week----------

In [ ]:
tripdata_final %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% 
  summarise(average_ride_length = mean(ride_length)) %>% 
  arrange(member_casual, weekday)  %>% 
  ggplot(aes(x = weekday, y = average_ride_length, fill = member_casual)) +
  geom_col(width=0.5, position = "dodge") + 
  labs(title ="Average ride length by Members and Casual riders Vs. Day of the week")


STEP 5 :  EXPORT SUMMARY FILE FOR FURTHER ANALYSIS

In [ ]:
write.csv(tripdata_final,"../capstone_cyclistic.csv", row.names = FALSE)

**CONCLUSION**

**FINDINGS**
* At present, around 43 percent of users are annual members and the rest is casual riders.
* Members have the bigger volume of data, except on saturday and sunday. On the weekend, casuals riders have the most data points.
* Casuals riders have more ride length (ride duration) than members.
* Busiest season is summer. There is an increase in number of rides in the weekends with a bigger volume of bikers in the afternoon.
* Members have a bigger preference for classic bikes, followed by electric bike.
* Members have a more fixed use for bikes for routine activities. Where as casual rider's usage is different, mostly all activiy in the weekend.

**SUGGESTIONS**
1. Offer additional benefits like priority for annual members over casual riders during rush hours of the day/week, less waiting time and preferred bike type.
2. Launch recreation centres near popular stations in order to attract riders. And a preference or discounts for members over casual riders.
3. Discount for members for reffering others and to those who completes 5 years to show we value and respect our clients.



Thanks for reading and I hope you like it.
Please give your valuable feedback.